# Inputs

* `path_to_entity_pool`: Path to `entity_pool.pkl` (See `Hard Negative Mining.ipynb`)
* `correct`: Correct entities per sample
* `files`: Filename per sample
* `pred`: Predicted entity per sample
* `scores`: Score of the predicted entity per sample
* `threshold`: NIL mention detection threshold

Scores are printed throughout the notebook.

In [ ]:
import numpy as np
import pickle
import json

In [ ]:
#List
correct = [] 
#List
files = []
#List
pred = []
#List
scores= [] 
#Flaot between [0,1]
threshold = 
path_to_entity_pool = "entity_pool.pkl"

In [ ]:
with open(path_to_entity_pool,"rb") as f:
    entity_pool = pickle.load(f)

In [ ]:
def get_thresholded_preds(th,pred,scores):
    thresholded_preds = []
    for i in range(len(pred)):
        if scores[i]>=th:
            thresholded_preds.append(pred[i])
        else:
            thresholded_preds.append(None)
    return thresholded_preds

In [ ]:
ctr = 0
thresholded_preds =  get_thresholded_preds(th,pred,scores)
for i in range(len(correct)):
    if thresholded_preds[i] is None and correct[i] is None:
        ctr+=1
    elif correct[i] is not None and str(thresholded_preds[i]) in entity_pool[correct[i]]:
        ctr+=1
print('Micro Average Accuracy: ',np.round((100*ctr)/len(correct),2),'%\n')

In [ ]:
#Group predictions by files for macro calc
grouped_preds = dict()
grouped_scores = dict()
grouped_correct = dict()
for i in range(len(files)):
    try:
        grouped_preds[files[i]].append(pred[i])
        grouped_scores[files[i]].append(scores[i])
        grouped_correct[files[i]].append(correct[i])
    except KeyError:
        grouped_preds[files[i]] = [pred[i]]
        grouped_scores[files[i]] = [scores[i]]
        grouped_correct[files[i]] = [correct[i]]

In [ ]:
#Calculate macro avg acc 
all_accuracies = []
for k in grouped_preds.keys():
    this_preds = grouped_preds[k]
    this_scores = grouped_scores[k]
    this_correct = grouped_correct[k]
    ctr = 0
    thresholded_preds =  get_thresholded_preds(th,this_preds,this_scores)
    for i in range(len(this_correct)):
        if thresholded_preds[i] is None and this_correct[i] is None:
            ctr+=1
        elif this_correct[i] is not None and  str(thresholded_preds[i]) in entity_pool[this_correct[i]]:
            ctr+=1
    all_accuracies.append(ctr/len(this_correct))
print('Macro Average Accuracy: ',np.round((100*(np.mean(all_accuracies))),2),'%\n')

In [ ]:
#Calculate EE precision

ee_precs = []

total_nil_predictions = 0
correct_nil_predictions = 0
thresholded_preds =  get_thresholded_preds(th,pred,scores)
for i in range(len(thresholded_preds)):
    if thresholded_preds[i] == None:
        total_nil_predictions+=1
        if correct[i] == None:
            correct_nil_predictions+=1
try:
    print('EE Precision: ',np.round((100*correct_nil_predictions)/total_nil_predictions,2),'%\n')
    ee_precs.append(np.round((100*correct_nil_predictions)/total_nil_predictions,2))
except ZeroDivisionError:
    print('EE Precision: 0','%\n')
    ee_precs.append(0.)

In [ ]:
#Calculate EE recall
ee_recs = []

total_nil_mentions = 0
correct_nil_mentions = 0
thresholded_preds =  get_thresholded_preds(th,pred,scores)
for i in range(len(correct)):
    if correct[i] == None:
        total_nil_mentions+=1
        if thresholded_preds[i] == None:
            correct_nil_mentions+=1
print('EE Recall: ',np.round((100*correct_nil_mentions)/total_nil_mentions,2),'%\n')
ee_recs.append(np.round((100*correct_nil_mentions)/total_nil_mentions,2))

In [ ]:
#Calculate InKB Precision
inkb_precs = []

total_mentions = 0
correct_mentions = 0
thresholded_preds =  get_thresholded_preds(th,pred,scores)
for i in range(len(thresholded_preds)):
    if thresholded_preds[i] is not None:
        total_mentions+=1
        if correct[i] is not None:
            if  str(thresholded_preds[i]) in entity_pool[correct[i]]:
                correct_mentions+=1
print('InKB Precision: ',np.round((100*correct_mentions)/total_mentions,2),'%\n')
inkb_precs.append(np.round((100*correct_mentions)/total_mentions,2))

In [ ]:
#Calculate InKB Recall
inkb_recs = []

total_mentions = 0
correct_mentions = 0
thresholded_preds =  get_thresholded_preds(th,pred,scores)
for i in range(len(correct)):
    if correct[i] is not None:
        total_mentions+=1
        if  str(thresholded_preds[i]) in entity_pool[correct[i]]:
            correct_mentions+=1
print('InKB Recall: ',np.round((100*correct_mentions)/total_mentions,2),'%\n')
inkb_recs.append(np.round((100*correct_mentions)/total_mentions,2))

In [ ]:
def get_f1(p,r):
    return (2*p*r)/(p+r)

In [ ]:
print("EE F1: ",np.round(get_f1(ee_precs[0],ee_recs[0]),2))
print("InKB F1: ",np.round(get_f1(inkb_precs[0],inkb_recs[0]),2))
print("\n")